### Вертикальная перестановка

В  шифре вертикальной перестановки реализуется перестановка  считываемых  столбцов  матрицы текста в  соответствии  с  ключом. Если ключ - это слово, то используется расположение букв ключа в алфавите.

In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from random import randint

def clear_paragraphs(ps:list, start=14, stop=16):
    return ''.join([i.text for i in ps[start:stop]]).replace('\xad', '').replace('\xa0', '')

def test_text(url='https://spacemorgue.com/matter-matters/'):
    '''
    Возвращает тестовый текст длиной 2408 символов
    '''
    soup = bs(requests.get(url).text, 'html.parser')
    return clear_paragraphs(soup.select_one('article').find_all('p'))

def prepare(text, specials={',':'ЗПТ','.':'ТЧК', ' ':'', '—':'ТИРЕ', '«':'КВЧ', '»':'КВЧ'}):
    '''
    Возвращает текст, удаляя пробельные символы в тексте и заменяя пунктуацию телеграфными обозначениями
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

def comp(text, specials={'ЗПТ':',','ТЧК':'.','ТИРЕ':'—','КВЧ':'"'}):
    '''
    Возвращает текст, заменяя телеграфные обозначения пунктуацией
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

al = [chr(i) for i in range(ord('А'), ord('Я') + 1) if i is not ord('Ё')]

In [2]:
def order(key):
    for i, ch in zip(range(len(key)), sorted(list(key))): # нумеруем сортированный по алфавиту список символов
        key = key.replace(ch, str(i)) # заменяем символ его номером
    return [int(i) for i in key]

In [71]:
def vertical_encrypt(plaintext, al, key):
    assert len(key) == len(set(key)), 'The key contains not unique symbols'
    al_internal = al.copy()
    al_internal.append('0')
    shape = [len(plaintext) // len(key), len(key)]
    #if len(plaintext) // len(key) > len(key):
    #    shape = [len(plaintext) // len(key), len(key)]
    #else:
    #    shape = [len(key), len(plaintext) // len(key)]
    if shape[0] * shape[1] < len(plaintext):
        shape[0] += shape[1] # добавить строку

    # дополняем текст пустыми символами для создания таблицы с требуемым числом столбцов
    if len(plaintext) % len(key):
        plaintext += al_internal[-1] * ((shape[0] * shape[1]) % len(plaintext))
        
    # создаем таблицу в виде матрицы и транспонируем для простоты обращения к столбцам:
    shape = (len(plaintext) // len(key), len(key))
    table = np.matrix(list(prepare(plaintext))).reshape(shape).T
    ciphertext = list()
    
    for i in order(key):
        ciphertext.append(table[i].tolist()[0])
    # преобразуем матрицу с переставленными столбцами в исходное положение и уберем пустой символ
    return ''.join(np.matrix(ciphertext).T.flatten().tolist()[0]).replace(al_internal[-1], '')

In [72]:
vertical_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al, 'БЛИНДАЖ')

'ЕКЯИВНСЗТКОПЙТИТЕЗТЧАТЕТНВПЧИУЛЗСИИАЧТКЕНТ'

In [75]:
def vertical_decrypt(ciphertext, al, key):
    al_internal = al.copy()
    al_internal.append('0')
    shape = [len(ciphertext) // len(key), len(key)]
    #if len(ciphertext) // len(key) > len(key):
    #    shape = [len(ciphertext) // len(key), len(key)]
    #else:
    #    shape = [len(key), len(ciphertext) // len(key)]

    if shape[0] * shape[1] < len(ciphertext):
        shape[0] += shape[1]
    if len(ciphertext) % len(key):
        ciphertext += al_internal[-1] * ((shape[0] * shape[1]) % len(ciphertext))
    table = np.matrix(list(ciphertext)).reshape(shape).T

    plaintext = list()
    key_l = order(key)
    for i in range(len(key)):
        j = key_l.index(i)
        plaintext.append(table[j].tolist()[0])
    return comp(''.join(np.matrix(plaintext).T.flatten().tolist()[0]).replace(al_internal[-1], ''))

In [76]:
vertical_decrypt('ЕКЯИВНСЗТКОПЙТИТЕЗТЧАТЕТНВПЧИУЛЗСИИАЧТКЕНТ', al, 'БЛИНДАЖ')

IndexError: index 6 is out of bounds for axis 0 with size 6

In [7]:
text = prepare(test_text().upper().replace('-', '—').replace('X', '').replace('I', ''))

In [78]:
vertical_encrypt(text, al, 'СЕНТЯБРЬ')

'НЕЛЕННИЙТЯИАИАСТЕТИСАСЧКЧПРИНЯИНССТНВОЬОВПРНСАИОИТВЛСЯФОУФСЮООККЦЦЕИКНПЮЛУЗЬОААНЗСВИЛЙЯССЖНТЗООЬТТКОАПОРАБЫПТЯЛОАРЯСОЕНПНВЛИМЯЕЕТОНИППЯЯЯСТНОООНЪКОЮКЙННЧИИКОЦТДИОЛТЛПНЕСНАЛЖЬЯОЬОСМЖНТОЗТВНКЕОИПУТРАНЬИЗАЛЕАНИКЗАЛАОТИТВОВОНРИЗВКНЕИИОНТЭКРМИСЕОЛЬЙОАНФЕМЫЛНРНИЙЙНПИЕОРОИНСИЧНТКПТООЗВТНОЙЕНРВШИЯПЧНЯРИОПОЖААРДБТСЫИЕОТЧЗПЬРЕТЯАЛЬКЮОЗЛСАЕЯИЧТЛАЬВПСШЗУЛЕЦОПКЕЙДСЛОАОЕВНЕЛОТТЬСЬДАНЙИЛЕОИХБТШСЫКЙТПВИЧОООРЕБТНОЕГАНЕОЩИААУЛНКЗЬЯХСЗЙЫВЕВПООЯИЗЛХТСПМЕИОЛЩЬУШОЮЧЯОБСИЕЪНОЬСЖОТЛНЕПОДНЕВЕТЕМЕИИАРЗЛОПНАВТЖВТЕРООВОМЯЗОЕВРИДЕЕХЖНИКБЪТВОЕИИОСПВНТОТЕЧЗСЛЕАСБОЕЬОЙРЕЗНФЛЕЫИССОКЕОФИЕОСДТПЛСИИЯРМВТЕИТЕАЬАРЛОЫЕБТНСЫТЯЗПРИПОЮЖДЩЕОАИИРУЕАДГМАЕРЛНТИЬБЕСЫИЫОТДВВЕОЯИБЛЕЕОНЛСЖРХСИСЫЕЙИБЫЙПОТЗЭПРТМТИОЕОВМЕССНОАЖЗТВНПУМОТИСГЛНЮДАЛЛИБЬДАИТЮТХРКХООЧОШСЗВТОИЕНТПТОУЗЧЛОБЕГСЧЛОСЕТОТВАСЗИТКМОИОНОНТВПЕОЕНОСВМЖТТЧИЫЦАСХВЕТИИВОЛВЛИОНДНЛТЧКОУТКМТЕЗРЗЖПАВЫЕЕОНЦТОЫЕМОВКПЫЕНЗТНТПБОДНТПООМНАУЫОМЗНАЛЫЗКЬХВУКЗТВОПТМЕСОИЮВССОТЕЙБВТНЫЕПНЙМАИБЦИВРИЧИРАТТЕСЧТУКЛОТБАГОРЭАДЯВИМУХТДМТРАЕИАКРАТИМВСКСВТИУМЕЗКОГУКЗОТАЬВАРЗЫТАМИЧЫЭЛИЕТФЕЫАФКНЖОЗИОМГ

In [77]:
vertical_decrypt(vertical_encrypt(text, al, 'СЕНТЯБРЬ'), al, 'СЕНТЯБРЬ')

'НЕЛИНЕЙНАЯИСТАТИСТИЧЕСКАЯПРИЧИННОСТЬСНОВАПРИВНОСЯТВФИЛОСОФСКУЮКОНЦЕПЦИЮКАУЗАЛЬНОЙСВЯЗИСЛОЖНОСТЬ,КОТОРАЯБЫЛАПОТЕРЯНАСПОЯВЛЕНИЕМПОНЯТИЯПОСТОЯННОЙКОНЪЮНКЦИИ.ДОПОЛНИТЕЛЬНАЯСЛОЖНОСТЬМОЖЕТВОЗНИКНУТЬПРИАНАЛИЗЕКАТАЛИЗАТОРОВИВОЗНИКНОВЕНИИЭКСТРЕМАЛЬНОЙФОРМЫНЕЛИНЕЙНОЙПРИЧИННОСТИ,ВКОТОРОЙВНЕШНЯЯПРИЧИНАПОРОЖДАЕТСОБЫТИЕ,ЧЬЯРОЛЬЗАКЛЮЧАЕТСЯЛИШЬВЗАПУСКЕЦЕЛОЙПОСЛЕДОВАТЕЛЬНОСТИДАЛЬНЕЙШИХСОБЫТИЙ.ПОВТОРНОЕОБОГАЩЕНИЕКАУЗАЛЬНЫХСВЯЗЕЙИПОЗВОЛЯЕТСИХПОМОЩЬЮЛУЧШЕОБЪЯСНИТЬСЛОЖНОЕПОВЕДЕНИЕМАТЕРИАЛОВ,НОВТОЖЕВРЕМЯВОЗРОЖДЕНИЕИХОБЪЕКТИВНОСТИПОВЛЕЧЕТЗАСОБОЙСЕРЬЕЗНЫЕФИЛОСОФСКИЕПОСЛЕДСТВИЯ—МАТЕРИАЛЬНЫЕСОБЫТИЯ,ПОРОЖДАЮЩИЕДРУГИЕМАТЕРИАЛЬНЫЕСОБЫТИЯВВИДЕБОЛЕЕСЛОЖНЫХСЕРИЙСОБЫТИЙ,ПРИЭТОМСОВСЕМНЕВАЖНО,СМОГУТЛИЛЮДИНАБЛЮДАТЬИХ.ХОРОШОИЗВЕСТНО,ЧТОЛУЧБЕЛОГОСВЕТАСОСТОИТИЗКОМПОНЕНТОВМНОЖЕСТВАЧИСТЫХЦВЕТОВИЛИДЛИНВОЛН.КТОМУЖЕ,РАЗНЫЕЦВЕТОВЫЕКОМПОНЕНТЫ,ПОДОБНОТОНАММУЗЫКАЛЬНЫХЗВУКОВ,ИМЕЮТСВОЙСОБСТВЕННЫЙТЕМПВИБРАЦИИ—ЧАСТОТУ.БЛАГОДАРЯЭТИМДВУМХАРАКТЕРИСТИКАМСВЕТИЗВУКМОГУТОКАЗЫВАТЬРАЗЛИЧИМЫЕЭФФЕКТЫНАМОЗГЖИВОТНЫХИЧЕЛОВЕКА—ЭФФЕКТЫ,КОТОРЫЕМОГУТИСП

### Решетка Кардано

Используется  трафарет  из  прямоугольного  листа  клетчатой  бумаги  размером $2m*2k$ клеток. 

В трафарете вырезано  $m*k$ клеток так, что при наложении его на чистый лист бумаги того же размера четырьмя возможными способами его вырезы полностью покрывают всю площадь листа. Буквы  сообщения  последовательно  вписываются  в  вырезы  трафарета  (по строкам,  в  каждой  строке  слева  направо)  при  каждом  из  четырех  его  возможных положений в заранее установленном порядке.

Пустые клетки заполняются случайными символами.

Решетку представим в виде матрицы, состоящей из нулей (пропуски) и единиц (вырезанные клетки):

In [291]:
key = np.matrix([
    [0,0,1,0,0,1,1,0,0,1],
    [0,1,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,1,0,0],
    [0,1,0,0,0,1,0,0,0,1],
    [1,0,0,0,1,0,1,1,0,0],
    [0,1,0,0,0,0,0,0,0,0],
      ])
key

matrix([[0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [392]:
def cardano_encrypt(plaintext, al, key : np.matrix):
    size = key.shape[0] * key.shape[1]
    assert size == 4 * len(np.where(key == 1)[0]), 'Cage is not cover all cells'
    
    plaintext = prepare(plaintext)
    # заполним пустые клетки псевднослучайными символами
    while len(plaintext) <= size:
        plaintext = plaintext + al[randint(0, len(al) - 1)]
    plaintext = list(plaintext)[::-1]
    # заполним таблицу по трафарету
    flip_results = list()
    for flip in [
        key,
        np.fliplr(key), # отражение трафарета слева-направо
        np.flipud(key), # отражение сверху-вниз
        np.flipud(np.fliplr(key))
    ]:
        flip_result = list()
        for row in flip.A:
            new_row = list()
            for cell in row:
                if cell:
                    cell = plaintext.pop()
                new_row.append(cell)
            flip_result.append(new_row)
        flip_results.append(np.matrix(flip_result))
    # соединим кусочки таблицы в строку шифротекста
    ciphertext = list()
    for i in range(key.shape[0]):
        for j in range(key.shape[1]):
            for flip in flip_results:
                if flip[i,j] != '0':
                    ciphertext.append(flip[i,j])
                    break
    return ''.join(ciphertext)

In [393]:
cardano_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al, key)

'ИСНТАЕВЕГСИЯЬФЛЕУЗТФЯНЗКЩАПИТЕТЙЮТВЗПТЧПТЧТЕКНТОФИДЧКВТАЮНИК'

In [395]:
def cardano_decrypt(ciphertext, al, key : np.matrix):
    ciphertext = np.matrix(list(ciphertext)).reshape(key.shape)
    plaintext = list()
    for flip in [
        key,
        np.fliplr(key),
        np.flipud(key),
        np.flipud(np.fliplr(key))
    ]:
        plaintext.extend(ciphertext[np.nonzero(flip)].A[0].tolist())
    return comp(''.join(plaintext).rsplit('ТЧК')[0] + 'ТЧК')

In [396]:
cardano_decrypt('ИСНТАЕВЕГСИЯЬФЛЕУЗТФЯНЗКЩАПИТЕТЙЮТВЗПТЧПТЧТЕКНТОФИДЧКВТАЮНИК', al, key)

'НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'